In [ ]:
# ✅ Step 1: Install Required Libraries
!pip install transformers datasets sentencepiece torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# ✅ Step 2: Upload Your Insurance QA CSV
from google.colab import files
uploaded = files.upload()

Saving insurance_intents.csv to insurance_intents.csv


In [ ]:
# ✅ Step 3: Load & Preprocess Dataset
import pandas as pd
from datasets import Dataset

# Load dataset
df = pd.read_csv("insurance_intents.csv")  # Update with actual filename
df = df.rename(columns={"input": "question", "output": "answer"})  # Rename for clarity
df = df.dropna()  # Remove missing values

# Convert to Hugging Face Dataset format
dataset = Dataset.from_pandas(df)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments

# Use t5-large instead of t5-base
model_checkpoint = "t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# Update training arguments for better performance
training_args = TrainingArguments(
    output_dir="./insurance-t5-large",
    num_train_epochs=5,  # Increased from 3 to 5
    per_device_train_batch_size=8,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    evaluation_strategy="no",
    save_strategy="epoch"
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# ✅ Step 5: Improved Preprocessing Function
def preprocess(example):
    input_text = f"insurance query: {example['question'].strip()}"
    target_text = example["answer"].strip()

    # Tokenize input and output
    model_inputs = tokenizer(
        input_text,
        max_length=128,
        truncation=True,
        padding="max_length"  # Ensures consistent tensor sizes
    )

    labels = tokenizer(
        target_text,
        max_length=128,
        truncation=True,
        padding="max_length"
    )["input_ids"]

    # Replace padding tokens with -100 to ignore in loss calculation
    labels = [-100 if token == tokenizer.pad_token_id else token for token in labels]

    model_inputs["labels"] = labels
    return model_inputs


In [ ]:
# ✅ Step 6: Apply Tokenization
dataset = dataset.map(preprocess, remove_columns=["question", "answer"])

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
# ✅ Step 7: Train-Validation Split
dataset = dataset.train_test_split(test_size=0.1)  # 90% training, 10% validation
train_dataset = dataset["train"]
val_dataset = dataset["test"]


In [ ]:
# ✅ Step 8: Load Model & Training Setup
from transformers import AutoModelForSeq2SeqLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq

In [ ]:
# Load pre-trained T5 model
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
# Data collator handles padding during training
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./insurance-t5-large",  # Better to reflect model
    per_device_train_batch_size=4,      # Reduce if memory issues with t5-large
    per_device_eval_batch_size=4,
    num_train_epochs=5,                 # More training time helps generalization
    learning_rate=3e-5,                 # Lower LR for large models is common
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="loss",       # Choose best model by loss
    greater_is_better=False             # Lower loss = better
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# ✅ Step 9: Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

<ipython-input-12-8c4e9716c464>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# ✅ Step 10: Train the Model
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: naveen1207 (naveen1207-guvi-geek-networks) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.050500,0.002244
2,0.012700,0.000058
3,0.002900,0.000020
4,0.001200,0.000011
5,0.002800,0.000010


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=565, training_loss=0.14798396207820788, metrics={'train_runtime': 1107.4002, 'train_samples_per_second': 2.032, 'train_steps_per_second': 0.51, 'total_flos': 1217839104000000.0, 'train_loss': 0.14798396207820788, 'epoch': 5.0})

In [ ]:
# ✅ Step 11: Save Model & Tokenizer
model.save_pretrained("/content/insurance_t5_model")
tokenizer.save_pretrained("/content/insurance_t5_model")

NameError: name 'model' is not defined

In [ ]:
# ✅ Step 12: Zip & Download Model
from zipfile import ZipFile
!zip -r insurance_t5_model.zip /content/insurance_t5_model

from google.colab import files
files.download("/content/insurance_t5_model.zip")

  adding: content/insurance_t5_model/ (stored 0%)
  adding: content/insurance_t5_model/spiece.model (deflated 48%)
  adding: content/insurance_t5_model/generation_config.json (deflated 29%)
  adding: content/insurance_t5_model/model.safetensors (deflated 10%)
  adding: content/insurance_t5_model/tokenizer_config.json (deflated 95%)
  adding: content/insurance_t5_model/tokenizer.json (deflated 74%)
  adding: content/insurance_t5_model/special_tokens_map.json (deflated 86%)
  adding: content/insurance_t5_model/config.json (deflated 62%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Move the zip file to your Google Drive
!cp /content/insurance_t5_model.zip /content/drive/MyDrive/


Mounted at /content/drive
cp: cannot stat '/content/insurance_t5_model.zip': No such file or directory
